In [7]:
import gdown
import tarfile
import numpy as np
import cv2
from skimage import io
import matplotlib.pyplot as plt

# Função de segmentação automática ajustada
def automatic_segmentation(image):
    # Converte a imagem para escala de cinza
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # Aplica o Otsu's Thresholding
    _, binary_mask = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Verifica se o fundo é branco ou preto
    object_pixels = np.sum(binary_mask == 255)
    background_pixels = np.sum(binary_mask == 0)

    # Se a maioria dos pixels for branca, assumimos que o fundo é branco e precisamos inverter
    if object_pixels > background_pixels:
        binary_mask = cv2.bitwise_not(binary_mask)

    # Converte a máscara para binário (0 ou 1)
    binary_mask = binary_mask // 255  # Divide por 255 para transformar os valores em 0 e 1
    return binary_mask

# Função para processar imagens de um arquivo tar.gz
def process_images_from_tar_gz(file_like_object, fruitFolders):
    # Abre o arquivo tar.gz diretamente do byte stream
    with tarfile.open(fileobj=file_like_object, mode='r:gz') as tar:
        # Obtém todos os membros (arquivos) do tar.gz
        members = tar.getmembers()

        # Filtra e ordena os diretórios em ordem alfabética
        directories = sorted([m for m in members if m.isdir()], key=lambda x: x.name)

        for directory in directories:
            fruit = directory.name.split('/')[-1]  # Obtém o nome da fruta do caminho do diretório
            if fruit not in fruitFolders:
                continue  # Pula se a fruta não estiver na lista

            # Obtém arquivos de imagem no diretório
            images = sorted([m for m in members if m.name.startswith(directory.name) and m.isfile()], key=lambda x: x.name)

            for img_member in images:
                file = tar.extractfile(img_member)
                if file:
                    # Lê a imagem
                    image = io.imread(file)
                    if image is None:
                        continue  # Ignora se não for uma imagem

                    # Segmentação automática
                    binary_mask = automatic_segmentation(image)

                    # Converte BGR para RGB para exibição
                    if len(image.shape) == 3 and image.shape[2] == 3:
                        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    else:
                        image_rgb = image  # Mantém as imagens em escala de cinza como estão

                    # Exibe a imagem original e a segmentação
                    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
                    ax[0].imshow(image_rgb)
                    ax[0].set_title("Imagem Original")
                    ax[0].axis('off')

                    ax[1].imshow(binary_mask, cmap='gray')
                    ax[1].set_title("Segmentação Automática (Ground truth)")
                    ax[1].axis('off')

                    plt.show()

# URL para baixar o arquivo do Google Drive via gdown (use apenas o ID do arquivo)
file_id = '17yStUzdDOAlOXr4ZAghBqgEHXB-NTeqt'
download_url = f"https://drive.google.com/uc?id={file_id}"

# Baixar o arquivo .tar.gz
output = 'file.tar.gz'
gdown.download(download_url, output, quiet=True)

# Abrir o arquivo baixado e processar
with open(output, 'rb') as f:
    process_images_from_tar_gz(f, ['apple', 'araticum', 'avocado', 'banana', 'butia', 'khaki', 'mango', 'orange', 'star_fruit', 'ugli_fruit'])



Output hidden; open in https://colab.research.google.com to view.